In [1]:
# Sentiment Analysis

import pandas as pd
df = pd.read_json(r"C:\Users\Mouni\Downloads\renttherunway_final_data.json", lines=True)
df.sample(10)

,fit,user_id,bust size,item_id,weight,rating,rented for,review_text,body type,review_summary,category,height,size,age,review_date
178594,fit,421524,NaN,2351339,135lbs,6.0,vacation,I rented this coat to wear at Sundance in Park...,athletic,Cute coat but a bit boxy,coat,"5' 5""",8,29.0,"January 28, 2016"
125986,fit,183019,NaN,799723,160lbs,10.0,formal affair,This dress is true to size and photographs ext...,athletic,Rehearsal dinner dress!,dress,"5' 8""",16,31.0,"September 6, 2017"
28572,small,249905,36b,187164,130lbs,10.0,party,I liked the dress and the fit was great! You ...,athletic,"Did not wear, color washed me out",dress,"5' 5""",14,44.0,"March 7, 2014"
97919,fit,957319,34c,2795602,145lbs,10.0,work,"Fits true to size -- typically wear size 8, me...",athletic,Cute/comfortable sweater - easy to dress up or...,sweater,"5' 8""",14,26.0,"November 6, 2017"
174003,large,129370,36d+,2955734,NaN,8.0,work,"i thought this would run small since it was ""d...",hourglass,"full, flippy, and adorable!!",skirt,"5' 7""",24,31.0,"February 21, 2016"
83079,large,985145,34d,994031,140lbs,6.0,work,"I got this dress to wear to work, and maybe to...",hourglass,Interesting but awkward dress,dress,"5' 8""",12,51.0,"August 3, 2017"
108350,small,845148,34d,158028,156lbs,8.0,formal affair,Great fit on my curves but way too short to we...,full bust,Nie but too Short,gown,"5' 10""",21,37.0,"February 9, 2016"
52076,fit,673509,34c,512791,160lbs,8.0,work,This dress is thick sweater material which is ...,hourglass,"nice, warm dress",dress,"5' 3""",14,29.0,"October 27, 2016"
127037,large,350614,32b,960463,110lbs,8.0,other,"Great dress, awesome color and fit. The 0 was ...",hourglass,Loved wearing this dress for my godson's bapti...,dress,"5' 5""",1,29.0,"September 30, 2015"
12686,fit,231087,32a,1333481,100lbs,10.0,everyday,I loved this dress. It is extremely comfortabl...,straight & narrow,Basically a fancy bathrobe,dress,"5' 7""",4,28.0,"November 7, 2017"


In [2]:
from sklearn import preprocessing
import nltk
nltk.download('opinion_lexicon')
from nltk.corpus import opinion_lexicon
from nltk.tokenize import word_tokenize


print('Total number of words in opinion lexicon', len(opinion_lexicon.words()))
print('Examples of positive words in opinion lexicon',opinion_lexicon.positive()[:10])
print('Examples of negative words in opinion lexicon',opinion_lexicon.negative()[:10])

Total number of words in opinion lexicon 6789
Examples of positive words in opinion lexicon ['a+', 'abound', 'abounds', 'abundance', 'abundant', 'accessable', 'accessible', 'acclaim', 'acclaimed', 'acclamation']
Examples of negative words in opinion lexicon ['2-faced', '2-faces', 'abnormal', 'abolish', 'abominable', 'abominably', 'abominate', 'abomination', 'abort', 'aborted']


[nltk_data] Downloading package opinion_lexicon to
[nltk_data]     C:\Users\Mouni\AppData\Roaming\nltk_data...
[nltk_data]   Package opinion_lexicon is already up-to-date!


In [3]:

##Let's create a dictionary which we can use for scoring our review text

nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Mouni\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:

df.rename(columns={"review_summary": "text"}, inplace=True)
pos_score = 1
neg_score = -1
word_dict = {}

# Adding the positive words to the dictionary
for word in opinion_lexicon.positive():
    word_dict[word] = pos_score

# Adding the negative words to the dictionary
for word in opinion_lexicon.negative():
    word_dict[word] = neg_score

In [5]:
def bing_liu_score(text):
    sentiment_score = 0
    bag_of_words = word_tokenize(text.lower())
    for word in bag_of_words:
        if word in word_dict:
            sentiment_score += word_dict[word]
    return sentiment_score

In [6]:
df['text'].fillna('no review', inplace=True)
df['Bing_Liu_Score'] = df['text'].apply(bing_liu_score)

In [7]:
df[['rating',"text", 'Bing_Liu_Score']].head(10)

,rating,text,Bing_Liu_Score
0,10.0,So many compliments!,0
1,10.0,I felt so glamourous!!!,0
2,10.0,It was a great time to celebrate the (almost) ...,2
3,8.0,Dress arrived on time and in perfect condition.,1
4,10.0,Was in love with this dress !!!,1
5,8.0,Traditional with a touch a sass,-1
6,10.0,LITERALLY THE CUTEST DRESS EVER,0
7,10.0,"Great dress, beautifully made. I received lot...",2
8,10.0,Great for black tie event!,1
9,10.0,This dress was everything! It was perfect for ...,1


In [8]:
df.groupby('rating').agg({'Bing_Liu_Score':'mean'})

,Bing_Liu_Score
rating,
2.0,0.078394
4.0,0.302401
6.0,0.708984
8.0,1.141653
10.0,1.255426
